In [ ]:
####### RF 3 PAIR 2021
import numpy as np
import rasterio
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import joblib

# مسیر فایل‌ها
base_path = r'D:\hagh\google drive/'
high_res1_path = f'{base_path}LANDSATBCN/LST_Landsat8_20210117.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 1
high_res2_path = f'{base_path}LANDSATBCN/LST_Landsat8_20210202.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 2
high_res3_path = f'{base_path}LANDSATBCN/LST_Landsat8_20210306.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 3
low_res1_path = f'{base_path}MODISBCN/LST_MODIS_Aqua_2021_01_17.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 1
low_res2_path = f'{base_path}MODISBCN/LST_MODIS_Aqua_2021_02_02.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 2
low_res3_path = f'{base_path}MODISBCN/LST_MODIS_Aqua_2021_03_06.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 3
output_path = f'{base_path}OUTPUTLST2021/RF3OUTPUT20210202.tif'  # مسیر ذخیره نتیجه
model_path = f'{base_path}OUTPUTLST2021/RF_model_2021.pkl'  # مسیر ذخیره مدل

# تابع بارگذاری تصاویر
def load_image(image_path):
    with rasterio.open(image_path) as src:
        image = src.read(1)  # فقط باند اول را می‌خوانیم
        transform = src.transform
        profile = src.profile
    return image, transform, profile

# بارگذاری تصاویر
high_res1_image, _, _ = load_image(high_res1_path)
high_res2_image, _, _ = load_image(high_res2_path)
high_res3_image, _, _ = load_image(high_res3_path)
low_res1_image, _, _ = load_image(low_res1_path)
low_res2_image, _, _ = load_image(low_res2_path)
low_res3_image, _, _ = load_image(low_res3_path)

# آماده‌سازی داده‌ها برای مدل
X = np.stack((
    high_res1_image.flatten(),
    high_res3_image.flatten(),
    low_res1_image.flatten(),
    low_res2_image.flatten(),
    low_res3_image.flatten()
), axis=-1)  # ورودی‌ها (ویژگی‌ها)
y = high_res2_image.flatten()  # خروجی (برچسب‌ها) تصویر با رزولوشن بالا در زمان 2

# تقسیم داده‌ها به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# ساخت مدل جنگل تصادفی
modelrf = RandomForestRegressor(n_estimators=100, random_state=42)
modelrf.fit(X_train, y_train)

# ذخیره مدل آموزش‌دیده
joblib.dump(modelrf, model_path)
print(f"Model saved to {model_path}")

# پیش‌بینی و ارزیابی مدل
y_pred = modelrf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# پیش‌بینی برای تمامی داده‌ها
high_res_pred_image = modelrf.predict(X).reshape(high_res2_image.shape)

# ذخیره تصویر پیش‌بینی‌شده
def save_image(image, reference_transform, reference_profile, output_path):
    with rasterio.open(output_path, 'w', **reference_profile) as dst:
        dst.write(image, 1)

# ذخیره خروجی پیش‌بینی‌شده
_, ref_transform, ref_profile = load_image(high_res2_path)
save_image(high_res_pred_image, ref_transform, ref_profile, output_path)

# نمایش تصویر پیش‌بینی‌شده
plt.imshow(high_res_pred_image, cmap='hot')
plt.colorbar()
plt.title("Predicted High-Resolution Image")
plt.show()

print(f"Predicted image saved to {output_path}")


In [ ]:
#########پیش بینی برای 2021 داده B
import numpy as np
import rasterio
import joblib
import matplotlib.pyplot as plt


base_path = r'D:\hagh\google drive/'

model_path = f'{base_path}OUTPUTLST2021\RF_model_2021.pkl'
high_res1_path = f'{base_path}LANDSATACN\LST_Landsat8_20210117.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 1
# مسیر تصویر با رزولوشن بالا در تاریخ 2
high_res3_path = f'{base_path}LANDSATACN\LST_Landsat8_20210306.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 3
low_res1_path = f'{base_path}MODISACN/LST_MODIS_Aqua_2021_01_17.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 1
low_res2_path = f'{base_path}MODISACN/LST_MODIS_Aqua_2021_02_02.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 2
low_res3_path = f'{base_path}MODISACN/LST_MODIS_Aqua_2021_03_06.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 3
output_path = f'{base_path}OUTPUTLST2021/RFDATACN_PredictedA_Output_20210202.tif'  # مسیر ذخیره نتیجه

# تابع بارگذاری تصاویر
def load_image(image_path):
    with rasterio.open(image_path) as src:
        image = src.read(1)  # فقط باند اول را می‌خوانیم
        transform = src.transform
        profile = src.profile
    return image, transform, profile

# بارگذاری مدل ذخیره‌شده
modelrf = joblib.load(model_path)
print("Model loaded successfully.")

# بارگذاری تصاویر جدید
low_res1_image, _, _ = load_image(low_res1_path)
low_res2_image, _, _ = load_image(low_res2_path)
low_res3_image, _, _ = load_image(low_res3_path)
high_res1_image, _, _ = load_image(high_res1_path)
high_res3_image, ref_transform, ref_profile = load_image(high_res3_path)

# آماده‌سازی داده‌ها برای پیش‌بینی
X_new = np.stack((
    high_res1_image.flatten(),
    high_res3_image.flatten(),
    low_res1_image.flatten(),
    low_res2_image.flatten(),
    low_res3_image.flatten()
), axis=-1)

# پیش‌بینی با مدل ذخیره‌شده
high_res_pred_image = modelrf.predict(X_new).reshape(high_res1_image.shape)

# ذخیره تصویر پیش‌بینی‌شده
def save_image(image, reference_transform, reference_profile, output_path):
    with rasterio.open(output_path, 'w', **reference_profile) as dst:
        dst.write(image, 1)

save_image(high_res_pred_image, ref_transform, ref_profile, output_path)
print(f"Predicted image saved to {output_path}")

# نمایش تصویر پیش‌بینی‌شده
plt.imshow(high_res_pred_image, cmap='hot')
plt.colorbar()
plt.title("Predicted High-Resolution Image")
plt.show()


In [ ]:
###########  XGBoost ###########2016
import numpy as np
import rasterio
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import pickle  # برای ذخیره مدل



# مسیر فایل‌ها
base_path = r'D:\hagh\google drive/'
high_res1_path = f'{base_path}LANDSATACN/LST_Landsat8_20160205.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 1
high_res2_path = f'{base_path}LANDSATACN/LST_Landsat8_20160308.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 2
high_res3_path = f'{base_path}LANDSATACN/LST_Landsat8_20160425.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 3
low_res1_path = f'{base_path}MODISACN/LST_MODIS_Aqua_2016_02_05.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 1
low_res2_path = f'{base_path}MODISACN/LST_MODIS_Aqua_2016_03_08.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 2
low_res3_path = f'{base_path}MODISACN/LST_MODIS_Aqua_2016_04_25.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 3
output_path = f'{base_path}OUTPUTLST2016/XGBoost3OUTPUT20160308.tif'  # مسیر ذخیره نتیجه
model_path = f'{base_path}OUTPUTLST2016/XGBoost_model_2016.pkl'  # مسیر ذخیره مدل

# تابع بارگذاری تصاویر
def load_image(image_path):
    with rasterio.open(image_path) as src:
        image = src.read(1)  # باند اول
        transform = src.transform
        profile = src.profile
    return image, transform, profile

# بارگذاری تصاویر
high_res1_image, _, _ = load_image(high_res1_path)
high_res2_image, _, _ = load_image(high_res2_path)
high_res3_image, _, _ = load_image(high_res3_path)
low_res1_image, _, _ = load_image(low_res1_path)
low_res2_image, _, _ = load_image(low_res2_path)
low_res3_image, _, _ = load_image(low_res3_path)

# آماده‌سازی داده‌ها برای مدل
X = np.stack((high_res1_image.flatten(), high_res3_image.flatten(),
              low_res1_image.flatten(), low_res2_image.flatten(),
              low_res3_image.flatten()), axis=-1)  # ویژگی‌ها
y = high_res2_image.flatten()  # خروجی (تصویر رزولوشن بالا در زمان 2)

# تقسیم داده‌ها به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# ساخت مدل XGBoost
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train, y_train)

# ذخیره مدل
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"Model saved to {model_path}")

# پیش‌بینی و ارزیابی مدل
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# پیش‌بینی برای کل تصویر
high_res_pred_image = model.predict(X).reshape(high_res2_image.shape)

# ذخیره تصویر پیش‌بینی‌شده
def save_image(image, reference_transform, reference_profile, output_path):
    reference_profile.update(dtype=rasterio.float32, count=1)
    with rasterio.open(output_path, 'w', **reference_profile) as dst:
        dst.write(image.astype(np.float32), 1)

# ذخیره خروجی
_, ref_transform, ref_profile = load_image(high_res2_path)
save_image(high_res_pred_image, ref_transform, ref_profile, output_path)

print(f"Predicted image saved to {output_path}")

# نمایش تصویر پیش‌بینی‌شده
plt.imshow(high_res_pred_image, cmap='hot')
plt.colorbar()
plt.title("Predicted High-Resolution Image with XGBoost")
plt.show()


In [ ]:
###########پیش بینی 2016 ###############
import numpy as np
import rasterio
import pickle
import matplotlib.pyplot as plt

base_path = r'D:\hagh\google drive/'

model_path = f'{base_path}OUTPUTLST2016\XGBoost_model_2016.pkl'
high_res1_path = f'{base_path}LANDSATBCN\LST_Landsat8_20160205.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 1
# مسیر تصویر با رزولوشن بالا در تاریخ 2
high_res3_path = f'{base_path}LANDSATBCN\LST_Landsat8_20160425.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 3
low_res1_path = f'{base_path}MODISBCN/LST_MODIS_Aqua_2016_02_05.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 1
low_res2_path = f'{base_path}MODISBCN/LST_MODIS_Aqua_2016_03_08.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 2
low_res3_path = f'{base_path}MODISBCN/LST_MODIS_Aqua_2016_04_25.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 3
output_path = f'{base_path}OUTPUTLST2016/XGBoostDATACN_Predicted_Output_20160308.tif'  # مسیر ذخیره نتیجه


# تابع بارگذاری تصاویر
def load_image(image_path):
    with rasterio.open(image_path) as src:
        image = src.read(1)  # باند اول
        transform = src.transform
        profile = src.profile
    return image, transform, profile

# بارگذاری تصاویر
high_res1_image, _, _ = load_image(high_res1_path)
high_res3_image, _, _ = load_image(high_res3_path)
low_res1_image, _, _ = load_image(low_res1_path)
low_res2_image, _, _ = load_image(low_res2_path)
low_res3_image, _, _ = load_image(low_res3_path)

# آماده‌سازی داده‌ها برای پیش‌بینی
X_new = np.stack((high_res1_image.flatten(), high_res3_image.flatten(),
                  low_res1_image.flatten(), low_res2_image.flatten(),
                  low_res3_image.flatten()), axis=-1)

# بارگذاری مدل ذخیره‌شده
with open(model_path, 'rb') as model_file:
    model = pickle.load(model_file)

# پیش‌بینی داده‌های جدید
high_res_pred_image = model.predict(X_new).reshape(high_res1_image.shape)

# ذخیره تصویر پیش‌بینی‌شده
def save_image(image, reference_transform, reference_profile, output_path):
    reference_profile.update(dtype=rasterio.float32, count=1)
    with rasterio.open(output_path, 'w', **reference_profile) as dst:
        dst.write(image.astype(np.float32), 1)

# ذخیره خروجی
_, ref_transform, ref_profile = load_image(high_res1_path)
save_image(high_res_pred_image, ref_transform, ref_profile, output_path)

# نمایش تصویر پیش‌بینی‌شده
plt.imshow(high_res_pred_image, cmap='hot')
plt.colorbar()
plt.title("Predicted High-Resolution Image")
plt.show()

print(f"Predicted image saved to {output_path}")


In [ ]:
##########XGB
import numpy as np
import rasterio
import glob
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib

# ⭕️ **مسیرها**
base_path = r'D:\hagh\google drive\OUTPUTLST2024'  
input_dir = f"{base_path}/STDFMB"  # مسیر تصاویر ورودی
target_path = f"{base_path}/LSTB_Landsat9_20240219.tif"  # مسیر تصویر هدف (Landsat LST)
model_path = f"{base_path}/GBM_model.pkl"  # مسیر ذخیره مدل
output_prediction_path = f"{base_path}/STDFMA/GBM_combined_A.tif"  # مسیر ذخیره خروجی پیش‌بینی‌شده

# ⭕️ **خواندن لیست تصاویر ورودی**
input_files = sorted(glob.glob(f"{input_dir}/*.tif"))  # خواندن تمام فایل‌های TIFF ورودی
if len(input_files) == 0:
    raise ValueError("No input images found!")

# ⭕️ **خواندن ویژگی‌های تصویر هدف برای تنظیمات ذخیره‌سازی**
with rasterio.open(target_path) as src:
    target_image = src.read(1).astype(np.float32)  # خواندن خروجی اصلی (Landsat LST)
    transform = src.transform  # اطلاعات مکانی تصویر
    profile = src.profile  # مشخصات تصویر
    height, width = src.shape  # ابعاد تصویر

# ⭕️ **خواندن و ترکیب تصاویر ورودی**
feature_stack = []  # لیست نگه‌داری ویژگی‌ها
for file in input_files:
    with rasterio.open(file) as src:
        image = src.read(1).astype(np.float32)  # خواندن باند اول
        
        # جایگزینی مقادیر NaN با میانگین تصویر
        nan_mask = np.isnan(image)
        if np.any(nan_mask):
            image[nan_mask] = np.nanmean(image)
        
        feature_stack.append(image.flatten())  # ذخیره مقدار پیکسل‌ها در لیست

# ⭕️ **ایجاد ماتریس ویژگی‌ها**
X = np.stack(feature_stack, axis=-1)  # ترکیب داده‌ها به‌صورت یک ماتریس ویژگی
y = target_image.flatten()  # مقدار هدف

# ⭕️ **تقسیم داده‌ها به آموزش و تست**
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# ⭕️ **ساخت مدل GBM و آموزش**
print("Training Gradient Boosting Model...")
model = GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# ⭕️ **ذخیره مدل**
joblib.dump(model, model_path)
print(f"Model saved to {model_path}")


In [ ]:
#############\پیش بینی 
########پیش بینی جدید 
import numpy as np
import rasterio
import glob
import joblib

# 🟢 **مسیرها**
base_path = r'D:\hagh\google drive\OUTPUTLST2024'  
input_dir = f"{base_path}/STDFMA"  # مسیر تصاویر جدید
model_path = f"{base_path}/GBM_model.pkl"  # مدل ذخیره‌شده
output_prediction_path = f"{base_path}/A_Predicted_GBM_COMBINED.tif"  # مسیر ذخیره خروجی جدید

# 🟢 **بارگذاری مدل ذخیره‌شده**
model = joblib.load(model_path)
print(f"Model loaded from {model_path}")

# 🟢 **خواندن لیست تصاویر جدید**
input_files = sorted(glob.glob(f"{input_dir}/*.tif"))  
if len(input_files) == 0:
    raise ValueError("No new input images found!")

# 🟢 **خواندن ویژگی‌های تصویر اول برای تنظیمات ذخیره‌سازی**
with rasterio.open(input_files[0]) as src:
    transform = src.transform
    profile = src.profile
    height, width = src.shape

# 🟢 **خواندن و ترکیب تصاویر ورودی جدید**
feature_stack = []
for file in input_files:
    with rasterio.open(file) as src:
        image = src.read(1).astype(np.float32)
        feature_stack.append(image.flatten())

# 🟢 **ایجاد ماتریس ویژگی‌ها برای پیش‌بینی**
X_new = np.stack(feature_stack, axis=-1)

# 🟢 **انجام پیش‌بینی**
print("Predicting new data...")
y_pred_new = model.predict(X_new).reshape(height, width)

# 🟢 **ذخیره خروجی پیش‌بینی‌شده**
profile.update(dtype=rasterio.float32)
with rasterio.open(output_prediction_path, 'w', **profile) as dst:
    dst.write(y_pred_new, 1)

print(f"New predicted image saved to {output_prediction_path}")



In [ ]:
######### DEPP FOREST  2 ############2022
import numpy as np
import rasterio
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib  # برای ذخیره مدل

# تابع بارگذاری تصاویر
def load_image(image_path):
    with rasterio.open(image_path) as src:
        image = src.read(1)  # فقط باند اول را می‌خوانیم
        transform = src.transform
        profile = src.profile
    return image, transform, profile

# مسیر فایل‌ها
base_path = r'D:\hagh\google drive/'
high_res1_path = f'{base_path}LANDSATBCN/LST_Landsat9_20220112.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 1
high_res2_path = f'{base_path}LANDSATBCN/LST_Landsat8_20220120.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 2
high_res3_path = f'{base_path}LANDSATBCN/LST_Landsat9_20220213.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 3
low_res1_path = f'{base_path}MODISBCN/LST_MODIS_Terra_2022_01_12.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 1
low_res2_path = f'{base_path}MODISBCN/LST_MODIS_Aqua_2022_01_20.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 2
low_res3_path = f'{base_path}MODISBCN/LST_MODIS_Aqua_2022_02_13.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 3
output_path = f'{base_path}OUTPUTLST2022/deep_forest_2_layer_20220120.tif'  # مسیر ذخیره نتیجه
model_path = f'{base_path}OUTPUTLST2022/deep_forest_2_layer_model.pkl'  # مسیر مدل آموزش‌داده‌شده

# بارگذاری تصاویر
high_res1_image, _, _ = load_image(high_res1_path)
high_res2_image, transform, profile = load_image(high_res2_path)
high_res3_image, _, _ = load_image(high_res3_path)
low_res1_image, _, _ = load_image(low_res1_path)
low_res2_image, _, _ = load_image(low_res2_path)
low_res3_image, _, _ = load_image(low_res3_path)

# آماده‌سازی داده‌ها برای مدل
X = np.stack((high_res1_image.flatten(), high_res3_image.flatten(),
              low_res1_image.flatten(), low_res2_image.flatten(), low_res3_image.flatten()), axis=-1)  # ورودی‌ها (ویژگی‌ها)
y = high_res2_image.flatten()  # خروجی (برچسب‌ها) تصویر با رزولوشن بالا در زمان 2

# تقسیم داده‌ها به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# مدل Random Forest برای لایه اول
rf1 = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf1.fit(X_train, y_train)

# پیش‌بینی لایه اول
X_train_pred1 = rf1.predict(X_train).reshape(-1, 1)
X_test_pred1 = rf1.predict(X_test).reshape(-1, 1)

# مدل Random Forest برای لایه دوم
rf2 = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf2.fit(np.hstack((X_train, X_train_pred1)), y_train)

# پیش‌بینی لایه دوم
y_pred = rf2.predict(np.hstack((X_test, X_test_pred1)))

# محاسبه خطای مدل
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# پیش‌بینی تصویر کامل
X_full_pred1 = rf1.predict(X).reshape(-1, 1)
y_full_pred = rf2.predict(np.hstack((X, X_full_pred1)))

# بازگرداندن به شکل تصویر
predicted_image = y_full_pred.reshape(high_res2_image.shape)

# ذخیره تصویر پیش‌بینی‌شده
with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(predicted_image.astype(rasterio.float32), 1)

print(f"Predicted image saved to: {output_path}")

# ذخیره مدل‌ها
joblib.dump(rf1, model_path.replace('.pkl', '_rf1.pkl'))
joblib.dump(rf2, model_path.replace('.pkl', '_rf2.pkl'))


In [ ]:
########پیش بینی 2022################### DEPP FOREST   2 ############2022
import numpy as np
import rasterio
import joblib  # برای بارگذاری مدل

# تابع بارگذاری تصاویر
def load_image(image_path):
    with rasterio.open(image_path) as src:
        image = src.read(1)  # فقط باند اول را می‌خوانیم
        transform = src.transform
        profile = src.profile
    return image, transform, profile

# مسیر فایل‌ها
base_path = r'D:\hagh\google drive/'
high_res1_path = f'{base_path}LANDSATACN\LST_Landsat9_20220112.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 1
high_res3_path = f'{base_path}LANDSATACN\LST_Landsat9_20220213.tif'  # مسیر تصویر با رزولوشن بالا در تاریخ 3
low_res1_path = f'{base_path}MODISACN\LST_MODIS_Terra_2022_01_12.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 1
low_res2_path = f'{base_path}MODISACN\LST_MODIS_Aqua_2022_01_20.tif'  # مسیر تصویر با رزولوشن پایین در تاریخ 2
low_res3_path = f'{base_path}MODISACN/LST_MODIS_Aqua_2022_02_13.tif'
output_path = f'{base_path}OUTPUTLST2022/deep_forestDATACN_2_layer_pridictA_20220120.tif'  # مسیر ذخیره نتیجه

# بارگذاری تصاویر
high_res1_image, _, _ = load_image(high_res1_path)
high_res3_image, transform, profile = load_image(high_res3_path)  # فقط High Res 1 و 3
low_res1_image, _, _ = load_image(low_res1_path)
low_res2_image, _, _ = load_image(low_res2_path)
low_res3_image, _, _ = load_image(low_res3_path)

# آماده‌سازی داده‌ها برای مدل
X_new = np.stack((high_res1_image.flatten(), high_res3_image.flatten(),
                  low_res1_image.flatten(), low_res2_image.flatten(), low_res3_image.flatten()), axis=-1)

# بارگذاری مدل‌های ذخیره‌شده
rf1 = joblib.load(f'{base_path}OUTPUTLST2022/deep_forest_2_layer_model_rf1.pkl')
rf2 = joblib.load(f'{base_path}OUTPUTLST2022/deep_forest_2_layer_model_rf2.pkl')

# پیش‌بینی لایه اول با مدل rf1
X_new_pred1 = rf1.predict(X_new).reshape(-1, 1)

# پیش‌بینی لایه دوم با استفاده از خروجی لایه اول و مدل rf2
y_new_pred = rf2.predict(np.hstack((X_new, X_new_pred1)))

# بازگرداندن پیش‌بینی‌ها به شکل تصویر
predicted_image_new = y_new_pred.reshape(low_res2_image.shape)  # شکل تصویر مشابه با Low Res 2

# ذخیره تصویر پیش‌بینی‌شده
with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(predicted_image_new.astype(rasterio.float32), 1)

print(f"Predicted image saved to: {output_path}")
